In [ ]:
using LinearAlgebra, SparseArrays

using gmsh
using WriteVTK

In [ ]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the y-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

In [ ]:
function gmsh_add_rectangle(mid, width, height, lc)
    geo = gmsh.model.geo;
    
    # Corner points
    p1 = geo.addPoint(mid[1] - width / 2, mid[2] - height / 2, 0, lc);
    p2 = geo.addPoint(mid[1] + width / 2, mid[2] - height / 2, 0, lc);
    p3 = geo.addPoint(mid[1] + width / 2, mid[2] + height / 2, 0, lc);
    p4 = geo.addPoint(mid[1] - width / 2, mid[2] + height / 2, 0, lc);
    points = [p1, p2, p3, p4];
    
    # Lines
    l1 = geo.addLine(p1, p2);
    l2 = geo.addLine(p2, p3);
    l3 = geo.addLine(p3, p4);
    l4 = geo.addLine(p4, p1);
    lines = [l1, l2, l3, l4];
    
    # Curve loop
    loop = geo.addCurveLoop(lines);
    
    return loop, lines, points;
end

In [ ]:
function gmsh_add_rectangle(mid, width, height, lc, radius)
    geo = gmsh.model.geo;
    
    # Corner points
    p1 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] - height / 2, 0, lc);
    p2 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] - height / 2, 0, lc);
    p3 = geo.addPoint(mid[1] + width / 2         , mid[2] - height / 2 + radius, 0, lc);
    p4 = geo.addPoint(mid[1] + width / 2         , mid[2] + height / 2 - radius, 0, lc);
    p5 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] + height / 2, 0, lc);
    p6 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] + height / 2, 0, lc);
    p7 = geo.addPoint(mid[1] - width / 2         , mid[2] + height / 2 - radius, 0, lc);
    p8 = geo.addPoint(mid[1] - width / 2         , mid[2] - height / 2 + radius, 0, lc);
    
    c1 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] - height / 2 + radius, 0, 1);
    c2 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] - height / 2 + radius, 0, 1);
    c3 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] + height / 2 - radius, 0, 1);
    c4 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] + height / 2 - radius, 0, 1);
    points = [p1, p2, p3, p4, p5, p6, p7, p8];
    
    # Lines
    l1 = geo.addLine(p1, p2)
    l2 = geo.addCircleArc(p2, c2, p3)
    l3 = geo.addLine(p3, p4)
    l4 = geo.addCircleArc(p4, c3, p5)
    l5 = geo.addLine(p5, p6)
    l6 = geo.addCircleArc(p6, c4, p7)
    l7 = geo.addLine(p7, p8)
    l8 = geo.addCircleArc(p8, c1, p1)
    lines = [l1, l2, l3, l4, l5, l6, l7, l8]
    
    # Curve loop
    loop = geo.addCurveLoop(lines);
    
    return loop, lines, points;
end

In [ ]:
gmsh.model.add("transformer")
geo = gmsh.model.geo;

## Enclosure
enclosure_lp, enclosure_lines, _ = gmsh_add_rectangle([0, 0.5 * (hencl1 + hencl2)], wencl, hencl1 - hencl2, lc1)

## Core 
core_lp, core_lines, _  = gmsh_add_rectangle([0, 0], wcore, hcore, lc1)     # Core outline
cgap1_lp, cgap1_lines, _ = gmsh_add_rectangle([-mgap, 0], wgap, hgap, lc2, 1.5e-2)   # Gap left
cgap2_lp, cgap2_lines, _ = gmsh_add_rectangle([+mgap, 0], wgap, hgap, lc2, 1.5e-2)   # Gap right

core_lines = union(core_lines, cgap1_lines, cgap2_lines);

## HV windings
xm = mgap + wgap / 2 + (wcore / 2 - mgap - wgap / 2) / 2;

hv1l_lp, hv1l_lines, _ = gmsh_add_rectangle([-xm - mwhv, 0], wwhv, hwhv, lc3)
hv1r_lp, hv1r_lines, _ = gmsh_add_rectangle([-xm + mwhv, 0], wwhv, hwhv, lc3)
hv2l_lp, hv2l_lines, _ = gmsh_add_rectangle([    - mwhv, 0], wwhv, hwhv, lc3)
hv2r_lp, hv2r_lines, _ = gmsh_add_rectangle([    + mwhv, 0], wwhv, hwhv, lc3)
hv3l_lp, hv3l_lines, _ = gmsh_add_rectangle([+xm - mwhv, 0], wwhv, hwhv, lc3)
hv3r_lp, hv3r_lines, _ = gmsh_add_rectangle([+xm + mwhv, 0], wwhv, hwhv, lc3)

hv_lines = union(hv1l_lines, hv1r_lines, hv2l_lines, hv2r_lines, hv3l_lines, hv3r_lines);

## LV windings
xm = mgap + wgap / 2 + (wcore / 2 - mgap - wgap / 2) / 2;

lv1l_lp, lv1l_lines, _ = gmsh_add_rectangle([-xm - mwlv, 0], wwlv, hwlv, lc4)
lv1r_lp, lv1r_lines, _ = gmsh_add_rectangle([-xm + mwlv, 0], wwlv, hwlv, lc4)
lv2l_lp, lv2l_lines, _ = gmsh_add_rectangle([    - mwlv, 0], wwlv, hwlv, lc4)
lv2r_lp, lv2r_lines, _ = gmsh_add_rectangle([    + mwlv, 0], wwlv, hwlv, lc4)
lv3l_lp, lv3l_lines, _ = gmsh_add_rectangle([+xm - mwlv, 0], wwlv, hwlv, lc4)
lv3r_lp, lv3r_lines, _ = gmsh_add_rectangle([+xm + mwlv, 0], wwlv, hwlv, lc4)

lv_lines = union(lv1l_lines, lv1r_lines, lv2l_lines, lv2r_lines, lv3l_lines, lv3r_lines);

## Surfaces
#geo.addPlaneSurface([enclosure_lp, core_lp, hv1l_lp, lv1l_lp, hv3r_lp, lv3r_lp], 1)
geo.addPlaneSurface([core_lp, cgap1_lp, cgap2_lp], 2)
geo.addPlaneSurface([cgap1_lp, hv1r_lp, lv1r_lp, hv2l_lp, lv2l_lp], 3)
geo.addPlaneSurface([cgap2_lp, hv2r_lp, lv2r_lp, hv3l_lp, lv3l_lp], 4)

geo.addPlaneSurface([hv1l_lp], 5)
geo.addPlaneSurface([hv1r_lp], 6)
geo.addPlaneSurface([hv2l_lp], 7)
geo.addPlaneSurface([hv2r_lp], 8)
geo.addPlaneSurface([hv3l_lp], 9)
geo.addPlaneSurface([hv3r_lp], 10)

geo.addPlaneSurface([lv1l_lp], 11)
geo.addPlaneSurface([lv1r_lp], 12)
geo.addPlaneSurface([lv2l_lp], 13)
geo.addPlaneSurface([lv2r_lp], 14)
geo.addPlaneSurface([lv3l_lp], 15)
geo.addPlaneSurface([lv3r_lp], 16)

geo.synchronize()

## Physical Groups
#geo.addPhysicalGroup(2, [1, 3, 4], 1)   # Transformer oil
geo.addPhysicalGroup(2, [2], 2)         # Core
geo.addPhysicalGroup(2, [5], 3)         # HV winding phase 1 left
geo.addPhysicalGroup(2, [6], 4)         # HV winding phase 1 right
geo.addPhysicalGroup(2, [7], 5)         # HV winding phase 2 left
geo.addPhysicalGroup(2, [8], 6)         # HV winding phase 2 right
geo.addPhysicalGroup(2, [9], 7)         # HV winding phase 3 left
geo.addPhysicalGroup(2, [10], 8)        # HV winding phase 3 right

geo.addPhysicalGroup(2, [11], 9)        # LV winding phase 1 left
geo.addPhysicalGroup(2, [12], 10)       # LV winding phase 1 right
geo.addPhysicalGroup(2, [13], 11)       # LV winding phase 2 left
geo.addPhysicalGroup(2, [14], 12)       # LV winding phase 2 right
geo.addPhysicalGroup(2, [15], 13)       # LV winding phase 3 left
geo.addPhysicalGroup(2, [16], 14)       # LV winding phase 3 right

geo.addPhysicalGroup(2, [5, 6, 7, 8, 9, 10], 15)      # HV windings
geo.addPhysicalGroup(2, [11, 12, 13, 14, 15, 16], 16) # LV windings

geo.addPhysicalGroup(1, enclosure_lines, 1)  # Enclosure boundary
geo.addPhysicalGroup(1, core_lines, 2)       # Core boundary
geo.addPhysicalGroup(1, hv_lines, 3)         # HV winding boundary
geo.addPhysicalGroup(1, lv_lines, 4)         # LV winding boundary

gmsh.model.setPhysicalName(2, 1, "Oil")
gmsh.model.setPhysicalName(2, 2, "Core")
gmsh.model.setPhysicalName(2, 15, "HV windings")
gmsh.model.setPhysicalName(2, 16, "LV windings")

gmsh.model.setPhysicalName(1, 1, "Enclosure")
gmsh.model.setPhysicalName(1, 2, "Core Boundary")
gmsh.model.setPhysicalName(1, 3, "HV Winding Boundary")
gmsh.model.setPhysicalName(1, 4, "LV Winding Boundary")

# Generate Mesh
geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/transformer_stedin_thermal.msh")

# Load Geometry

In [ ]:
# mesh_data struct
#  saves the nodes and elements of a gmsh geometry, for easy passing around in functions
struct mesh_data
    nnodes  # number of nodes
    xnode   # array of x coordinates
    ynode   # array of y coordinates
    
    # 2D mesh elements
    nelements   # number of elements
    e_group     # array containing the physical group number of each element
    elements    # conveniently structured connectivity array
    
    # 1D boundary elements
    nbnd_elem   # number of boundary elements
    bnd_elem    # conveniently structured connectivity array
    bnd_group   # array containing the physical group number of each element
end

# Loads nodes, elements, and element physical groups from gmsh and stores them in a mesh_data struct
function get_mesh_data()
    #..2/11 Get and sort the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    
    #..4/12 Get the mesh elements
    #..observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelements = length(element_ids[1])

    #..5/12 Create groups of elements for the subdomains
    #..for loop that creates a vector describing which physical group an element belongs to
    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
    ngroup3 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 3)
    ngroup4 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 4)
    ngroup5 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 5)
    ngroup6 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 6)
    ngroup7 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 7)
    ngroup8 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 8)
    ngroup9 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 9)
    ngroup10 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 10)
    ngroup11 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 11)
    ngroup12 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 12)
    ngroup13 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 13)
    ngroup14 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 14)
    ngroup15 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 15)
    ngroup16 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 16)
    e_group = zeros(1,nelements)
    
    elements = [zeros(Int, 3) for i in 1:nelements];
    
    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        
        # Store connectivity in a convenient format
        elements[element_id] = [node1_id, node2_id, node3_id];
        
        # Determine which physical group the element belongs to
        G1  = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Oil
        G2  = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Core
        G3  = sum(node1_id.== ngroup3[1])+sum(node2_id.== ngroup3[1])+sum(node3_id.== ngroup3[1]) # HV winding phase 1 left
        G4  = sum(node1_id.== ngroup4[1])+sum(node2_id.== ngroup4[1])+sum(node3_id.== ngroup4[1]) # HV winding phase 1 right
        G5  = sum(node1_id.== ngroup5[1])+sum(node2_id.== ngroup5[1])+sum(node3_id.== ngroup5[1]) # HV winding phase 2 left
        G6  = sum(node1_id.== ngroup6[1])+sum(node2_id.== ngroup6[1])+sum(node3_id.== ngroup6[1]) # HV winding phase 2 right
        G7  = sum(node1_id.== ngroup7[1])+sum(node2_id.== ngroup7[1])+sum(node3_id.== ngroup7[1]) # HV winding phase 3 left
        G8  = sum(node1_id.== ngroup8[1])+sum(node2_id.== ngroup8[1])+sum(node3_id.== ngroup8[1]) # HV winding phase 3 right
        G9  = sum(node1_id.== ngroup9[1])+sum(node2_id.== ngroup9[1])+sum(node3_id.== ngroup9[1]) # LV winding phase 1 left
        G10 = sum(node1_id.== ngroup10[1])+sum(node2_id.== ngroup10[1])+sum(node3_id.== ngroup10[1]) # LV winding phase 1 right
        G11 = sum(node1_id.== ngroup11[1])+sum(node2_id.== ngroup11[1])+sum(node3_id.== ngroup11[1]) # LV winding phase 2 left
        G12 = sum(node1_id.== ngroup12[1])+sum(node2_id.== ngroup12[1])+sum(node3_id.== ngroup12[1]) # LV winding phase 2 right
        G13 = sum(node1_id.== ngroup13[1])+sum(node2_id.== ngroup13[1])+sum(node3_id.== ngroup13[1]) # LV winding phase 3 left
        G14 = sum(node1_id.== ngroup14[1])+sum(node2_id.== ngroup14[1])+sum(node3_id.== ngroup14[1]) # LV winding phase 3 right
        
        if G1 == 3
            e_group[element_id] = 1;
        elseif G2 == 3
            e_group[element_id] = 2;
        elseif G3 == 3
            e_group[element_id] = 3;
        elseif G4 == 3
            e_group[element_id] = 4;
        elseif G5 == 3
            e_group[element_id] = 5;
        elseif G6 == 3
            e_group[element_id] = 6;
        elseif G7 == 3
            e_group[element_id] = 7;
        elseif G8 == 3
            e_group[element_id] = 8;
        elseif G9 == 3
            e_group[element_id] = 9;
        elseif G10 == 3
            e_group[element_id] = 10;
        elseif G11 == 3
            e_group[element_id] = 11;
        elseif G12 == 3
            e_group[element_id] = 12;
        elseif G13 == 3
            e_group[element_id] = 13;
        elseif G14 == 3
            e_group[element_id] = 14;
        end
    end
    
    
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(1);
    nbnd_elem = length(element_ids[1]);

    bnd_elem  = [zeros(Int, 2) for i in 1:nbnd_elem];
    bnd_group = zeros(1, nbnd_elem)

    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(1, 2);
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(1, 3);
    ngroup3 = gmsh.model.mesh.getNodesForPhysicalGroup(1, 4);

    for element_id in 1:nbnd_elem
        node1_id = element_connectivity[1][2*(element_id-1)+1]
        node2_id = element_connectivity[1][2*(element_id-1)+2]

        # Store connectivity in a convenient format
        bnd_elem[element_id] = [node1_id, node2_id];

        # Determine which boundary the element belongs to
        G1  = sum(node1_id.== ngroup1[1]) + sum(node2_id.== ngroup1[1]); # Core boundary
        G2  = sum(node1_id.== ngroup2[1]) + sum(node2_id.== ngroup2[1]); # HV winding boundary
        G3  = sum(node1_id.== ngroup3[1]) + sum(node2_id.== ngroup3[1]); # LV winding boundary

        if G1 == 2
            bnd_group[element_id] = 1;
        elseif G2 == 2
            bnd_group[element_id] = 2;
        elseif G3 == 2
            bnd_group[element_id] = 3;
        end
    end
    
    return mesh_data(nnodes, xnode, ynode, nelements, e_group, elements, nbnd_elem, bnd_elem, bnd_group)
end

# Thermal Modeling

## Heat Equation

$$ \rho c_p \frac{\partial T}{\partial t} - \nabla \cdot \left[ k \nabla T \right] = Q_v $$
where $\rho$ is the density $\mathrm{[kg/m^3]}$, $c_p$ is the specific heat capacity $\mathrm{[J/kg]}$, and $k$ is the thermal conductivity $\mathrm{[W/(m\cdot K)]}$. The heat sources are represented by $Q_v$.
This differential equation has to be combined with suitable boundary conditions:
- Dirichlet: imposes a specific temperature on a point, boundary, or domain.
- Neumann: imposes a heat flux on a point, boundary, or domain. This heat flux can describe processes such as radiation and convection.

## Convection
The cooling of the transformer is too complex to model exactly, since it involves convection to the oil which flows through the three-dimensional structure of the transformer (hence obtaining a three-dimensional temperature gradient). To simplify things, we model the oil with a constant temperature of $T_{ext} = 40\ \mathrm{^\circ C}$. The heat flux $q$ is
$$ q = h (T - T_{ext}) \qquad \mathrm{[W/m^2]} $$
where $h$ is the heat transfer coefficient $\mathrm{[W/(m^2\cdot K)]}$.

## Stationary Weak Form
Assuming that we are in steady-state (i.e., $\frac{\partial T}{\partial t} = 0$), the differential equation simplifies to
$$ -\nabla \cdot \left[ k \nabla T \right] = Q_v $$
Writing this in weak form we obtain
\begin{align*}
    -\int_\Omega \nabla \cdot \left[ k\nabla T \right] \phi_k d\Omega & = \int_\Omega Q_v \phi_k d\Omega \qquad 1 \le k \le N \\
    \int_\Omega k \nabla T \cdot \nabla \phi_k d\Omega & = \int_\Omega Q_v \phi_k d\Omega + \oint_\Gamma k \left[ \nabla T \cdot \mathbf{n} \right] \phi_k d\Gamma \qquad 1 \le k \le N
\end{align*}

## Implementation
The first two terms of the weak form can easily be implemented in their linear matrix equation form, as we have shown in other exercises. To impose the Neumann condition on the boundary of the transformer core and windings (which are in direct contact with the oil, and hence undergo convective cooling), we must add some elements to the linear system which correspond to the closed integral over the boundary $\Gamma$.

To facilitate this, we defined "one"-dimensional physical domains for each of the boundaries in Gmsh. These will allow us to implement the integration over the contour $\Gamma$. The next step is to realize that we can rewrite the integral:
$$ \oint_\Gamma [k\nabla T] \cdot \mathbf{n} \phi_k d\Gamma = \oint_\Gamma q_{conv} \phi_k d\Gamma = \oint_\Gamma h (T - T_{ext}) \phi_k d\Gamma $$
where $\Gamma$ is the boundary where convection happens. Next, we apply the trapezoidal rule for integrals over one dimension to obtain the matrix contribution of _boundary_ element $b_i$.
$$ g_{b_i} = \operatorname{length}(b_i) \begin{bmatrix}
    \frac{h}{2} (T_1 - T_{ext}) \\
    \frac{h}{2} (T_2 - T_{ext}) \\
\end{bmatrix} $$
where $T_1$ and $T_2$ are the temperatures (the unknown $u$) of the local nodes $1$ and $2$. Therefore, the contribution $g_{b_i}$ can be split into a contribution to $A$ and a contribution to $f$.
$$ A_{b_i} = \operatorname{length}(b_i) \frac{h}{2} \begin{bmatrix}
    1 & 0 \\
    0 & 1 \\
\end{bmatrix} \qquad f_{b_i} = \operatorname{length}(b_i) \frac{h}{2} \begin{bmatrix}
    1 \\ 1
\end{bmatrix} $$

In [ ]:
function fem2d(mshdata, source_elem, k_elem, Text)
    A = spzeros(mshdata.nnodes, mshdata.nnodes)
    f = zeros(mshdata.nnodes, 1)

    xnode = mshdata.xnode; ynode = mshdata.ynode;
    
    # Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        # Retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Compute local vector contribution floc of the current element
        floc = area_id/3*source_elem[element_id] * [1; 1; 1]

        # Compute local matrix contribution Aloc of the current element
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Emat[3,:] .= 0;
        Bloc = area_id*k_elem[element_id]*(transpose(Emat)*Emat);

        # Add local contribution to f and A
        f[nodes]       += floc;
        A[nodes,nodes] += Bloc;
    end

    # Handle the boundary conditions
    bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
    f[bnd_node_ids] .= Text;
    
    return A, f;
end

In [ ]:
function process(mshdata, u, source_elem, k_elem)
    Qx = zeros(mshdata.nelements);
    Qy = zeros(mshdata.nelements);
    
    xnode = mshdata.xnode; ynode = mshdata.ynode;
    
    # Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        # Get x and y coordinates of the three nodes
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Qx and Qy from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Qx[element_id] = -sum(c .* Emat[1,:]);
        Qy[element_id] = -sum(c .* Emat[2,:]);
    end
    
    return (Qx,Qy);
end

# Implementation

In [ ]:
gmsh.initialize()
gmsh.open("geo/transformer_stedin_thermal.msh")
mshdata = get_mesh_data();

In [ ]:
Text = 40 + 273.15;

# Source function 
Pcore = 1.3 * 7.65e3; # W/m^3
Pwdg  = 10e3;  # W/m^3
sourcefunction(group_id) = Pcore * (group_id == 2) + Pwdg * (group_id >= 3);
source_elem = map(sourcefunction, mshdata.e_group);

# Thermal conductivity
k_Oil   = 0.162;
k_Cu    = 401;
k_Steel = 45;
kfunction(group_id) = k_Oil * (group_id == 1) + k_Steel * (group_id == 2) + k_Cu * (group_id >= 3);
k_elem = map(kfunction, mshdata.e_group);

# Heat transfer coefficients
h1 = 50;
h2 = 5;

In [ ]:
A, f = fem2d(mshdata, source_elem, k_elem, Text);

xnode = mshdata.xnode; ynode = mshdata.ynode;

# Perform a loop over the boundary elements
for (element_id, nodes) in enumerate(mshdata.bnd_elem)
    # Retrieve global numbering of the local nodes of the current element
    node1_id = nodes[1]; node2_id = nodes[2];

    # Get x and y coordinates of the two nodes
    xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id];
    ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id];
    
    el_len = sqrt((xnode2 - xnode1)^2 + (ynode2 - ynode1)^2);
    
    if(mshdata.bnd_group[element_id] == 1)
        Aloc = el_len / 2 * h1 * [1 0; 0 1];
        floc = el_len / 2 * h1 * Text * [1; 1];
        
        A[nodes,nodes] += Aloc;
        f[nodes] += floc; 
    end
    if(mshdata.bnd_group[element_id] == 2 || mshdata.bnd_group[element_id] == 3)
        Aloc = el_len / 2 * h2 * [1 0; 0 1];
        floc = el_len / 2 * h2 * Text * [1; 1];
        
        A[nodes,nodes] += Aloc;
        f[nodes] += floc; 
    end
end

u = A \ f;

Q = process(mshdata, u, source_elem, k_elem);

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer_thermal/transformer1.vtu", points, cells);

# Store data in the VTK file
vtkfile["T", VTKPointData()] = u .- 273.15;
vtkfile["Q", VTKCellData()]  = Q;

# Save the file
outfiles = vtk_save(vtkfile);

![Result: Temperature](images/transformer_thermal/transformer1_T.png)

![Result: Heat Flux](images/transformer_thermal/transformer1_Q.png)